In [1]:
import os
import sys

root_path = os.path.join(os.getcwd(), "..")
if root_path not in sys.path:
    sys.path.insert(0, root_path)

In [2]:
import matplotlib.pyplot as ptl

from src.config import Config
from src.dataset import DataLoader

In [3]:
data_loader = DataLoader(config=Config)
images = data_loader.extract_content_pdf(
    filepath=f"{Config.RAW_DATA_PATH}/sach-nhung-cay-thuoc-va-vi-thuoc-viet-nam-45-51.pdf"
)

In [4]:
images

[<PIL.PngImagePlugin.PngImageFile image mode=RGB size=2100x3071>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2059x3071>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2100x3071>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2100x3071>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2100x3071>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2100x3071>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2100x3071>]